# Dataset Preprocessing

In [62]:
import numpy as np
import pandas as pd
from rdkit import (Chem, RDLogger)
from rdkit.Chem import Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.ML.Descriptors import MoleculeDescriptors as md

from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score,
    roc_auc_score, confusion_matrix, classification_report, roc_curve
)
import xgboost as xgb
import matplotlib.pyplot as plt
import seaborn as sns

## Molecule, Descriptor, and Outlier Utility Functions

### `molecule_from_smiles(smiles)`
Converts a SMILES string into a cleaned **RDKit molecule object**, while temporarily silencing RDKit logs to avoid console spam.

**Process:**
1. Parse the SMILES into an RDKit molecule (`Chem.MolFromSmiles`).
2. Remove salts and keep the **largest fragment** using `LargestFragmentChooser`.
3. Re-sanitize the molecule to ensure validity.
4. Logging is muted during processing and restored afterward.

**Returns:**
- `(molecule, status)`  
  - `molecule`: RDKit molecule object or `None`  
  - `status`: `"succeed"`, `"failed"`, or `"error: <message>"`

---

### `calculate_descriptors(molecule)`
Calculates all available **1D and 2D molecular descriptors** using RDKit’s built-in descriptor list.

**Steps:**
1. Collect all descriptor names from `Descriptors._descList`.  
2. Use `MolecularDescriptorCalculator` to compute their values for the molecule.  
3. Return as a dictionary mapping *descriptor name → value*.

**Returns:**
- `dict`: `{ descriptor_name: value }`

---

### `outliers_iqr(df, factor=1.5)`
Applies the **Interquartile Range (IQR)** rule to cap extreme numeric values.

**Process:**
- Compute Q1 (25%) and Q3 (75%) for each column.  
- Define bounds: `[Q1 − 1.5×IQR, Q3 + 1.5×IQR]`.  
- Values beyond these limits are **clipped** to the nearest boundary.  
- Columns with zero IQR (flat values) are skipped.

**Purpose:**
Removes the influence of outliers **without deleting rows**, preserving dataset structure and stabilizing machine learning models.

In [63]:
def molecule_from_smiles(smiles):
    lg = RDLogger.logger()
    # Temporarily silence RDKit logs (Only critical)
    lg.setLevel(RDLogger.CRITICAL)
    try:
        # Extract molecule
        molecule = Chem.MolFromSmiles(smiles, sanitize=True)
        if molecule is None:
            return None, "failed"

        # Remove salts
        clean_molecule = rdMolStandardize.LargestFragmentChooser()
        molecule = clean_molecule.choose(molecule)

        # Sanitize molecule again to reflect changes
        Chem.SanitizeMol(molecule)
        return molecule, "succeed"
    except Exception as e:
        return None, f"error: {e}"
    finally:
        # re-enable logging afterward
        lg.setLevel(RDLogger.INFO)


def calculate_descriptors(molecule):
    # Get all descriptors (1D/2D)
    descriptor_names = []
    for descriptor, _ in Descriptors._descList:
        descriptor_names.append(descriptor)

    # Use descriptors to calculate values
    calculator = md.MolecularDescriptorCalculator(descriptor_names)
    descriptor_values = calculator.CalcDescriptors(molecule)

    # Create dictionary
    descriptors = dict(zip(descriptor_names, descriptor_values))
    return descriptors


def outliers_iqr(df, factor=1.5):
    df_copy = df.copy()
    for col in df_copy.columns:
        # Only for numeric columns, but clean_desc should already be numeric
        q1 = df_copy[col].quantile(0.25)
        q3 = df_copy[col].quantile(0.75)
        iqr = q3 - q1

        # If IQR is 0 - column is too flat → skip
        if iqr == 0:
            continue

        lower = q1 - factor * iqr
        upper = q3 + factor * iqr

        # Apply the IQR limits
        df_copy[col] = df_copy[col].clip(lower, upper)
    return df_copy

## Dataset Processing and Descriptor Cleaning Pipeline

This section takes the original *in chemico* dataset, turns SMILES into RDKit molecules, computes 1D/2D descriptors, cleans them, and produces two outputs:  
1) a **full** Excel report (original data + raw descriptors + molecule status),  
2) a **clean** CSV with ML-ready features.

### Configuration Variables
- **`ORIG_DATASET`** – path to the original Excel file with SMILES and labels  
- **`SKIP_ROWS`** – how many first rows in Excel to skip (non-data header)  
- **`TARGET`** – column name that holds the SMILES strings  
- **`FULL_OUTPUT_DATASET`** – Excel file with original data + all raw descriptors + status  
- **`ML_CLEAN_CSV`** – CSV file with cleaned numeric descriptors (to feed ML)  
- **`SIMILARITY_THRESHOLD`** – if ≥ this fraction of rows has the same value in a column, the column is dropped (e.g. `0.80` → drop columns where 80%+ values are identical)

---

### Workflow Overview

1. **Load dataset**  
   Read the original Excel file (`ORIG_DATASET`) and skip non-data rows (`SKIP_ROWS`).

2. **SMILES → RDKit molecules**  
   Iterate over the SMILES column (`TARGET`), convert each string using `molecule_from_smiles()`,  
   and record the build status (`"succeed"`, `"failed"`, `"error: ..."`) so failed rows don’t break the pipeline.

3. **Compute RDKit descriptors**  
   For each successfully built molecule, call `calculate_descriptors()` and collect all 1D/2D descriptor values into a list of dictionaries.

4. **Build descriptor table**  
   Turn the list of descriptor dictionaries into a single `pandas.DataFrame`, so **each descriptor is a column** and **each molecule is a row**.

5. **Descriptor cleaning**  
   - keep only **numeric** descriptor columns,  
   - replace `inf` / `-inf` with `NaN`,  
   - fill remaining `NaN` with the **column median**,  
   - drop **constant or quasi-constant** descriptors where ≥ `SIMILARITY_THRESHOLD` of values are the same,  
   - **cap outliers with IQR** (`outliers_iqr(..., factor=1.5)`) to limit extreme descriptor values without dropping rows.

6. **Merge and save**  
   - concatenate the **original dataset** with the **raw (uncleaned) descriptors** and add the `MoleculeStatus` column,  
   - save this “full report” to `FULL_OUTPUT_DATASET` (Excel),  
   - save the **cleaned descriptor matrix** to `ML_CLEAN_CSV` (CSV) for model training.

7. **Logging / summary**  
   Print how many rows/columns the processed data has,  
   list the descriptor columns that were dropped as (almost) constant,  
   and preview the first rows of both the full and the clean datasets.


In [64]:
# Dataset processing variables
ORIG_DATASET = "in_chemico_dataset.xlsx"
SKIP_ROWS = 1
TARGET = "SMILES code"
FULL_OUTPUT_DATASET = "in_chemico_dataset_processed.xlsx"
ML_CLEAN_CSV = "in_chemico_dataset_clean.csv"
# Same values threshold - tolerance
SIMILARITY_THRESHOLD = 0.80

# Load dataset and skip first row (Header)
dataset = pd.read_excel(ORIG_DATASET, engine="openpyxl", skiprows=SKIP_ROWS)

descriptor_rows = []
state_molecules = []
molecules = []

# Loop over the SMILES column
for smiles in dataset[TARGET].astype(str):
    # Convert SMILES to molecule
    molecule, state = molecule_from_smiles(smiles)
    state_molecules.append(state)
    molecules.append(molecule)

    # If molecule construction failed - empty placeholder
    if molecule is None:
        descriptor_rows.append({})
        continue

    # Calculate descriptors for each molecule
    descriptors = calculate_descriptors(molecule)
    descriptor_rows.append(descriptors)

# Convert list of dictionaries into dataframe
descriptor_data = pd.DataFrame(descriptor_rows)

# Take only numeric descriptor columns
numeric_columns = []
for col in descriptor_data.columns:
    if pd.api.types.is_numeric_dtype(descriptor_data[col]):
        numeric_columns.append(col)

# Keep only numeric columns
clean_desc = descriptor_data[numeric_columns].copy()

# Replace inf/-inf with NaN
for col in clean_desc.columns:
    clean_desc[col] = clean_desc[col].replace([np.inf, -np.inf], np.nan)

# Fill missing values with column medians
clean_desc = clean_desc.fillna(clean_desc.median())

# Drop constant and almost constant columns
constant_cols = []

for col in clean_desc.columns:
    top_freq = clean_desc[col].value_counts(normalize=True, dropna=False).max()
    if top_freq >= SIMILARITY_THRESHOLD:
        constant_cols.append(col)

if constant_cols:
    clean_desc = clean_desc.drop(columns=constant_cols)

# Remove outliers - IQR used
clean_desc = outliers_iqr(clean_desc, factor=1.5)

# Combine original dataset + descriptors (Dataset + descriptors)
output = pd.concat([dataset.reset_index(drop=True), descriptor_data.reset_index(drop=True)], axis=1)
output["MoleculeStatus"] = state_molecules

# Save everything to Excel
with pd.ExcelWriter(FULL_OUTPUT_DATASET, engine="openpyxl") as writer:
    # Full report with raw 1D/2D descriptors
    output.to_excel(writer, index=False, sheet_name="Descriptors")

# Also save clean descriptors to CSV
clean_desc.to_csv(ML_CLEAN_CSV, index=False)

# Log to console - full processed dataset
print(f"Rows: {len(output)}/Columns: {output.shape[1]}")
print("First rows of full processed dataset:")
print(output.head().to_string(index=False))
# Log to console - clean ML dataset
print(f"Rows: {len(clean_desc)}/Columns: {clean_desc.shape[1]}")
if constant_cols:
    print("\nDropped descriptor columns (constant or almost constant):", end=" ")
    print(", ".join(constant_cols))
else:
    print("\nNo constant or almost constant columns were dropped.")
print("First rows of clean dataset:")
print(clean_desc.head().to_string(index=False))
clean_desc.describe()

Rows: 162/Columns: 230
First rows of full processed dataset:
                          Name                                                                                              IUPAC name CAS registry number    Structure  Phototoxicity                                                      SMILES code                            Sources               Note    Unnamed: 8 Unnamed: 9  Unnamed: 10 Unnamed: 11  MaxAbsEStateIndex  MaxEStateIndex  MinAbsEStateIndex  MinEStateIndex      qed       SPS   MolWt  HeavyAtomMolWt  ExactMolWt  NumValenceElectrons  NumRadicalElectrons  MaxPartialCharge  MinPartialCharge  MaxAbsPartialCharge  MinAbsPartialCharge  FpDensityMorgan1  FpDensityMorgan2  FpDensityMorgan3  BCUT2D_MWHI  BCUT2D_MWLOW  BCUT2D_CHGHI  BCUT2D_CHGLO  BCUT2D_LOGPHI  BCUT2D_LOGPLOW  BCUT2D_MRHI  BCUT2D_MRLOW   AvgIpc  BalabanJ    BertzCT      Chi0     Chi0n     Chi0v      Chi1    Chi1n    Chi1v    Chi2n    Chi2v    Chi3n    Chi3v    Chi4n    Chi4v  HallKierAlpha           Ipc    K

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_C_O,fr_C_O_noCOO,fr_NH0,fr_NH1,fr_aniline,fr_benzene,fr_bicyclic,fr_ether,fr_halogen,fr_para_hydroxylation
count,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,...,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000,162.000000
mean,11.289986,11.289986,0.199259,-0.873010,0.609190,14.202418,311.895502,293.943328,311.432324,113.899691,...,0.827160,0.592593,1.216049,0.419753,0.472222,1.290123,0.617284,0.540123,0.595679,0.283951
std,2.392413,2.392413,0.194383,1.062014,0.208226,4.840864,125.061757,118.475676,124.762316,43.979804,...,0.794627,0.780632,1.381866,0.615316,0.731394,0.801258,0.804767,0.801742,0.867877,0.562483
min,6.162601,6.162601,0.000139,-2.978083,0.139518,6.000000,46.069000,40.021000,46.041865,20.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,10.145950,10.145950,0.055422,-1.293657,0.482905,10.857143,226.504000,214.130750,226.341233,88.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
50%,11.666796,11.666796,0.128362,-0.823232,0.649877,12.275362,313.788000,296.937500,313.098190,112.000000,...,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,12.801516,12.801516,0.284581,-0.170706,0.787633,16.678571,372.800500,350.898500,372.135966,137.500000,...,1.000000,1.000000,2.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,0.000000
max,15.645307,15.645307,0.628319,1.311296,0.893230,25.410714,592.245250,556.050125,590.828067,211.750000,...,2.500000,2.500000,5.000000,2.500000,2.500000,3.000000,2.500000,2.500000,2.500000,2.000000
